In [7]:
import pandas as pd
import ee # Earth Engine API
ee.Authenticate()
ee.Initialize()


Successfully saved authorization token.


EEException: Not signed up for Earth Engine or project is not registered. Visit https://developers.google.com/earth-engine/guides/access

In [3]:
crashsite_df = pd.read_csv('out/crashsite_df.csv')
crashsite_df.head()

,X,Y,OBJECTID,ACCIDENT_NO,ABS_CODE,ACCIDENT_STATUS,ACCIDENT_DATE,ACCIDENT_TIME,ALCOHOLTIME,ACCIDENT_TYPE,...,LGA_NAME_ALL,REGION_NAME_ALL,SRNS,SRNS_ALL,RMA,RMA_ALL,DIVIDED,DIVIDED_ALL,STAT_DIV_NAME,predicted_environmental_cluster
0,2444652.517,2369157.816,1,T20150013818,ABS to receive accident,Finished,2015-07-01 00:00:00+00:00,1899/12/30 05:15:00+00,Yes,Collision with a fixed object,...,GEELONG,SOUTH WESTERN REGION,NaN,NaN,Local Road,Local Road,Undivided,Undivided,Country,39
1,2507053.522,2427188.691,2,T20150013844,ABS to receive accident,Finished,2015-07-01 00:00:00+00:00,1899/12/30 11:30:00+00,No,Collision with a fixed object,...,WHITTLESEA,METROPOLITAN NORTH WEST REGION,C,C,Arterial Other,"Arterial Other,Local Road",Divided,"Undivided,Divided",Metro,56
2,2490706.473,2409439.682,3,T20150013854,ABS to receive accident,Finished,2015-07-01 00:00:00+00:00,1899/12/30 13:40:00+00,No,Collision with vehicle,...,MARIBYRNONG,METROPOLITAN NORTH WEST REGION,NaN,NaN,Local Road,Local Road,Undivided,Undivided,Metro,87
3,2524370.961,2379929.797,4,T20150013897,Non ABS accident,Finished,2015-07-02 00:00:00+00:00,1899/12/30 10:31:00+00,No,Struck Pedestrian,...,CASEY,METROPOLITAN SOUTH EAST REGION,NaN,NaN,NaN,NaN,NaN,NaN,Metro,46
4,2346439.879,2385852.164,5,T20150013907,ABS to receive accident,Finished,2015-07-02 00:00:00+00:00,1899/12/30 10:25:00+00,No,Vehicle overturned (no collision),...,CORANGAMITE,SOUTH WESTERN REGION,C,C,Arterial Other,Arterial Other,Undivided,Undivided,Country,123


In [5]:
# create a function to take 100*100m satellite images from Google Earth Engine
# and save them to a local folder


import urllib
from io import StringIO  
def get_satellite_images(df, folder):

    

    # loop through each row in the dataframe
    for index, row in df.iterrows():

        # get the coordinates from the dataframe
        lat = row['latitude']
        lon = row['longitude']

        # create a point object
        point = ee.Geometry.Point([lon, lat])

        # create a 100*100m square around the point
        square = point.buffer(50).bounds()

        # get the satellite image
        image = ee.ImageCollection('COPERNICUS/S2').filterBounds(square).filterDate('2019-01-01', '2019-12-31').median()

        # get the image url
        url = image.getThumbURL({'min': 0, 'max': 3000, 'bands': ['B4', 'B3', 'B2']})

        # save the image to a local folder
        urllib.request.urlretrieve(url, f'out/{folder}/{index}.jpg')
        break

# run the function
get_satellite_images(crashsite_df, 'crashsite_images')

EEException: Not signed up for Earth Engine or project is not registered. Visit https://developers.google.com/earth-engine/guides/access